data processing


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/gdrive/My Drive/CICIDS2017/dataset/cicids2017_merge.csv')

In [ ]:
# df = df [['Subflow Fwd Bytes', 
#          'Bwd Packet Length Std',
#          'Flow Duration', 'Fwd IAT Mean','Init_Win_bytes_forward',      
#     'Label']]
df = df [['Subflow Fwd Bytes',   
    'Label']]

In [ ]:
df = df[1:100000]

In [ ]:
df.shape

(99999, 2)

In [ ]:
df.columns[~(df != 0).any(axis=0)]

Index([], dtype='object')

In [ ]:
df 

,Subflow Fwd Bytes,Label
1,6,BENIGN
2,6,BENIGN
3,6,BENIGN
4,12,BENIGN
5,12,BENIGN
...,...,...
99995,24,DDoS
99996,24,DDoS
99997,24,DDoS
99998,30,DDoS


remove all zero

In [ ]:
df['Label'].value_counts().index

Index(['DDoS', 'BENIGN'], dtype='object')

In [ ]:
df = df.loc[:, (df != 0).any(axis=0)]

renove nan and inf

In [ ]:
df.replace([-np.inf, np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

select features

In [ ]:
df.columns.values

array(['Subflow Fwd Bytes', 'Label'], dtype=object)

In [ ]:
#df = df[['Bwd Packet Length Max', 'Fwd Packet Length Max', 'Bwd Packet Length Std','Bwd Packet Length Min',
 #      'Label']]

In [ ]:
df

,Subflow Fwd Bytes,Label
1,6,BENIGN
2,6,BENIGN
3,6,BENIGN
4,12,BENIGN
5,12,BENIGN
...,...,...
99995,24,DDoS
99996,24,DDoS
99997,24,DDoS
99998,30,DDoS


split test and train

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
seed = 1

In [ ]:
train_set, test_set = train_test_split(df, test_size=0.3, random_state=seed)

split data and target

In [ ]:
train_X = train_set.drop('Label', axis=1)
train_y = train_set[['Label']]
test_X = test_set.drop('Label', axis=1)
test_y = test_set[['Label']]

normalization

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
Scaler = StandardScaler()

In [ ]:
train_X = Scaler.fit_transform(train_X)
test_X = Scaler.transform(test_X)

In [ ]:
train_X.shape

(69999, 1)

encode

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
y_test_ada = test_y

model

In [ ]:
import sklearn.ensemble

In [ ]:
model = sklearn.ensemble.RandomForestClassifier(n_estimators=50)
model.fit(train_X, train_y.values.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
train_y.shape

(69999, 1)

In [ ]:
import tensorflow as tf

In [ ]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
]

metrics=METRICS

In [ ]:
y_pred=model.predict(test_X)

In [ ]:
def display_metrics(y_test, y_pred, label_names):
  print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

  print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
  print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
  print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

  print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
  print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
  print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

  print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
  print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
  print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

  print('\nClassification Report\n')
  print(classification_report(y_test, y_pred, target_names=label_names))

In [ ]:
def make_value2index(attacks):
    #make dictionary
    attacks = sorted(attacks)
    d = {}
    counter=0
    for attack in attacks:
        d[attack] = counter
        counter+=1
    return d
    

In [ ]:
def encode_label(Y_str):
    labels_d = make_value2index(np.unique(Y_str))
    Y = [labels_d[y_str] for y_str  in Y_str]
    Y = np.array(Y)
    return np.array(Y)

In [ ]:
labels_d = make_value2index(y_test_ada['Label'])

In [ ]:
#importing confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn import metrics
from sklearn.metrics import accuracy_score

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [ ]:
display_metrics(test_y, y_pred, labels_d)


Accuracy: 0.99

Micro Precision: 0.99
Micro Recall: 0.99
Micro F1-score: 0.99

Macro Precision: 0.99
Macro Recall: 0.99
Macro F1-score: 0.99

Weighted Precision: 0.99
Weighted Recall: 0.99
Weighted F1-score: 0.99

Classification Report

              precision    recall  f1-score   support

      BENIGN       1.00      0.98      0.99     11807
        DDoS       0.99      1.00      0.99     18193

    accuracy                           0.99     30000
   macro avg       0.99      0.99      0.99     30000
weighted avg       0.99      0.99      0.99     30000



LIME 

In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(test_X)

In [ ]:
test_X[0]

array([-0.22539861])

In [ ]:
len(shap_values)

30000

In [ ]:
shap_values[0]

.values =
array([[-0.38605837,  0.38605837]])

.base_values =
array([0.38605837, 0.61394163])

.data =
array([-0.22539861])

In [ ]:
shap_values

.values =
array([[[-0.38605837,  0.38605837]],

       [[ 0.61052112, -0.61052112]],

       [[ 0.61394163, -0.61394163]],

       ...,

       [[ 0.61394163, -0.61394163]],

       [[-0.38605837,  0.38605837]],

       [[ 0.61394163, -0.61394163]]])

.base_values =
array([[0.38605837, 0.61394163],
       [0.38605837, 0.61394163],
       [0.38605837, 0.61394163],
       ...,
       [0.38605837, 0.61394163],
       [0.38605837, 0.61394163],
       [0.38605837, 0.61394163]])

.data =
array([[-0.22539861],
       [-0.23182738],
       [-0.21061244],
       ...,
       [-0.20868381],
       [-0.22539861],
       [ 3.49910878]])